# ADA Project
## Modeling

## 1. ARIMA

##### Check for stationnarity
--> The data is indeed stationnary

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Perform ADF test on a given time series
def perform_adf_test(series):
    result = adfuller(series, autolag='AIC')  # 'AIC' will choose the best lag based on information criterion
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    print('Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value}')
    
for wolf_id, group in data.groupby('individual-id'):
    print(f'Performing ADF Test on Wolf {wolf_id}')
    print('Latitude:')
    perform_adf_test(group['location-lat'])
    print('\nLongitude:')
    perform_adf_test(group['location-long'])
    print('\n')


Performing ADF Test on Wolf B042
Latitude:
ADF Statistic: -6.662488058718176
p-value: 4.808048600575763e-09
Critical Values:
	1%: -3.4353174541055567
	5%: -2.863733732389869
	10%: -2.5679379527245407

Longitude:
ADF Statistic: -5.4954821031500884
p-value: 2.1282023732637278e-06
Critical Values:
	1%: -3.4353478262263777
	5%: -2.863747134166378
	10%: -2.567945089732423


Performing ADF Test on Wolf B045
Latitude:
ADF Statistic: -6.595741879291992
p-value: 6.932980470528559e-09
Critical Values:
	1%: -3.4330175441935666
	5%: -2.862718497145558
	10%: -2.5673973613128673

Longitude:
ADF Statistic: -6.210648821948962
p-value: 5.518401090190218e-08
Critical Values:
	1%: -3.4330273737125445
	5%: -2.8627228377894505
	10%: -2.567399672341618


Performing ADF Test on Wolf B065
Latitude:
ADF Statistic: -2.8951712084538768
p-value: 0.04591240861261048
Critical Values:
	1%: -3.431777037044106
	5%: -2.8621705835006916
	10%: -2.5671056625190722

Longitude:
ADF Statistic: -4.795992348179957
p-value: 5.5

##### ARIMA model

In [1]:
import pandas as pd
from pmdarima import auto_arima
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


# Load and preprocess the data
data = pd.read_csv('../Data/merged_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Filter for 'Wolf'
data = data[data['animal-type'] == 'Wolf']

# Initialize results dictionary
results = {}

# Loop over each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    split_index = int(len(group) * 0.8)
    train_data = group.iloc[:split_index]
    test_data = group.iloc[split_index:]

    # Fit ARIMA model on the training data for latitude and longitude
    model_lat = auto_arima(train_data['location-lat'], seasonal=False, stepwise=True, max_p=2, max_q=2, d=0, suppress_warnings=True)
    model_long = auto_arima(train_data['location-long'], seasonal=False, stepwise=True, max_p=2, max_q=2, d=0, suppress_warnings=True)
    
    # Predict on the training set to assess training accuracy
    train_preds_lat = model_lat.predict(n_periods=len(train_data))
    train_preds_long = model_long.predict(n_periods=len(train_data))

    # Predict on the test set
    test_preds_lat = model_lat.predict(n_periods=len(test_data))
    test_preds_long = model_long.predict(n_periods=len(test_data))

    # Compute accuracy metrics for training
    train_mae_lat = mean_absolute_error(train_data['location-lat'], train_preds_lat)
    train_rmse_lat = np.sqrt(mean_squared_error(train_data['location-lat'], train_preds_lat))
    train_mae_long = mean_absolute_error(train_data['location-long'], train_preds_long)
    train_rmse_long = np.sqrt(mean_squared_error(train_data['location-long'], train_preds_long))

    # Compute accuracy metrics for testing
    test_mae_lat = mean_absolute_error(test_data['location-lat'], test_preds_lat)
    test_rmse_lat = np.sqrt(mean_squared_error(test_data['location-lat'], test_preds_lat))
    test_mae_long = mean_absolute_error(test_data['location-long'], test_preds_long)
    test_rmse_long = np.sqrt(mean_squared_error(test_data['location-long'], test_preds_long))

    # Store results
    results[wolf_id] = {
        'training_accuracy': {'MAE_lat': train_mae_lat, 'RMSE_lat': train_rmse_lat, 'MAE_long': train_mae_long, 'RMSE_long': train_rmse_long},
        'test_accuracy': {'MAE_lat': test_mae_lat, 'RMSE_lat': test_rmse_lat, 'MAE_long': test_mae_long, 'RMSE_long': test_rmse_long},
        'test_predictions': (test_preds_lat, test_preds_long),
        'actual_test_data': (test_data['location-lat'].values, test_data['location-long'].values)
    }

# Display results for each wolf
for wolf_id, info in results.items():
    print(f"Wolf ID: {wolf_id}")
    print("Training Accuracy:", info['training_accuracy'])
    print("Test Accuracy:", info['test_accuracy'])


/var/folders/cd/f_ps01dx04n4r8lqhy1tjk5r0000gn/T/ipykernel_1409/713560876.py:9: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../Data/merged_data.csv')
/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/robin/opt/anaconda

Wolf ID: B042
Training Accuracy: {'MAE_lat': 0.054534872154125454, 'RMSE_lat': 0.06728349563637272, 'MAE_long': 0.10424698249834792, 'RMSE_long': 0.13881910161610486}
Test Accuracy: {'MAE_lat': 0.03294863390738822, 'RMSE_lat': 0.04313707612642559, 'MAE_long': 0.049257005633222606, 'RMSE_long': 0.07174941828103924}
292    51.447327
293    51.446131
294    51.444858
295    51.443568
296    51.442282
         ...    
361    51.381060
362    51.380396
363    51.379740
364    51.379090
365    51.378447
Length: 74, dtype: float64 292   -116.021439
293   -116.019160
294   -116.016918
295   -116.014713
296   -116.012545
          ...    
361   -115.926776
362   -115.926046
363   -115.925327
364   -115.924621
365   -115.923926
Length: 74, dtype: float64
Wolf ID: B045
Training Accuracy: {'MAE_lat': 0.057131966240784926, 'RMSE_lat': 0.08044129010990296, 'MAE_long': 0.06373506687863137, 'RMSE_long': 0.07469966681203581}
Test Accuracy: {'MAE_lat': 0.04419543012804172, 'RMSE_lat': 0.0587734975761546

/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/Users/robin/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/base/tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


#### Map of the predictions

In [7]:
import folium

# Function to create a map for a single wolf
def create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id):
    # Starting point for the map
    start_lat = wolf_data['location-lat'].iloc[0]
    start_long = wolf_data['location-long'].iloc[0]
    wolf_map = folium.Map(location=[start_lat, start_long], zoom_start=8)

    # Split the data into training and testing
    split_index = int(len(wolf_data) * 0.8)
    train_data = wolf_data.iloc[:split_index]
    test_data = wolf_data.iloc[split_index:]

    # Define the HTML code for a green cross
    html_green_cross = '''
    <div style="position: relative; width: 8px; height: 8px;">
        <div style="position: absolute; top: 50%; left: 0; transform: translate(0%, -50%); width: 100%; height: 2px; background-color: purple;"></div>
        <div style="position: absolute; top: 0; left: 50%; transform: translate(-50%, 0%); width: 2px; height: 100%; background-color: purple;"></div>
    </div>
    '''
    
    # Add training data points and draw lines between them
    train_points = [(row['location-lat'], row['location-long']) for idx, row in train_data.iterrows()]
    folium.PolyLine(train_points, color="blue", weight=2.5, opacity=1).add_to(wolf_map)
    for point in train_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue',
            popup='Train'
        ).add_to(wolf_map)

    # Add test data points and draw lines between them
    test_points = [(row['location-lat'], row['location-long']) for idx, row in test_data.iterrows()]
    folium.PolyLine(test_points, color="green", weight=2.5, opacity=1).add_to(wolf_map)
    for point in test_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='green',
            fill=True,
            fill_color='green',
            popup='Test'
        ).add_to(wolf_map)

    # Mark the first test observation with a green cross
    folium.Marker(
        location=test_points[0],
        icon=folium.DivIcon(html=html_green_cross),
        popup='First Test Observation'
    ).add_to(wolf_map)

    # Add prediction points and draw lines between them
    prediction_points = list(zip(test_preds_lat, test_preds_long))
    folium.PolyLine(prediction_points, color="red", weight=2.5, opacity=1).add_to(wolf_map)
    for idx, point in enumerate(prediction_points):
        folium.CircleMarker(
            location=point,
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            popup=f'Predicted: {test_data.index[idx]}'
        ).add_to(wolf_map)

    # Save the map
    wolf_map.save(f'../Visualisation/ARIMA/wolf_{wolf_id}_map.html')

# Iterate through each wolf and create a map
for wolf_id, info in results.items():
    test_preds_lat = info['test_predictions'][0]
    test_preds_long = info['test_predictions'][1]
    wolf_data = data[data['individual-id'] == wolf_id]
    create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id)


### Haversine accuracy of ARIMA

In [79]:
import numpy as np
from math import radians, cos, sin, asin, sqrt

# Haversine function definition
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance between two points 
    on the Earth (specified in decimal degrees).
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Initialize results dictionary outside of the loop
results = {}

# Loop over each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    split_index = int(len(group) * 0.8)
    test_data = group.iloc[split_index:]

    # Assuming test_preds_lat and test_preds_long are already defined for each wolf
    # Calculate the Haversine distance for each prediction in the test set
    distances = [
        haversine(pred_lon, pred_lat, act_lon, act_lat)
        for pred_lon, pred_lat, act_lon, act_lat in zip(test_preds_long, test_preds_lat, test_data['location-long'], test_data['location-lat'])
    ]

    # Calculate mean, median, MAE, and RMSE for the Haversine distances
    mean_distance_error = np.mean(distances)
    median_distance_error = np.median(distances)
    
    # Store results including Haversine distances
    results[wolf_id] = {
        'training_accuracy': {
            'MAE_lat': test_mae_lat,  # Storing them under training for example
            'RMSE_lat': test_rmse_lat,
            'MAE_long': test_mae_long,
            'RMSE_long': test_rmse_long
        },
        'test_accuracy': {
            'Mean_Haversine_Distance': mean_distance_error,
            'Median_Haversine_Distance': median_distance_error
        }
    }

# Print the results for each wolf
for wolf_id, info in results.items():
    print(f"Wolf ID: {wolf_id}")
    print("Traditional Accuracy Metrics:", {
        'MAE_lat': info['training_accuracy']['MAE_lat'],
        'RMSE_lat': info['training_accuracy']['RMSE_lat'],
        'MAE_long': info['training_accuracy']['MAE_long'],
        'RMSE_long': info['training_accuracy']['RMSE_long']
    })
    print("Test Accuracy (Haversine):", {
        'Mean_Haversine_Distance': info['test_accuracy']['Mean_Haversine_Distance'],
        'Median_Haversine_Distance': info['test_accuracy']['Median_Haversine_Distance']
    })

Wolf ID: B042
Traditional Accuracy Metrics: {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}
Test Accuracy (Haversine): {'Mean_Haversine_Distance': 27.132738455889378, 'Median_Haversine_Distance': 26.379128873400788}
Wolf ID: B045
Traditional Accuracy Metrics: {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}
Test Accuracy (Haversine): {'Mean_Haversine_Distance': 31.7936922809775, 'Median_Haversine_Distance': 31.596251955466627}
Wolf ID: B065
Traditional Accuracy Metrics: {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}
Test Accuracy (Haversine): {'Mean_Haversine_Distance': 42.087636286319, 'Median_Haversine_Distance': 41.4538188010044}
Wolf ID: B077
Traditional Accuracy Metrics: {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, '

## 2. NN

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error

# Load and preprocess the data
data = pd.read_csv('../Data/merged_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)
data = data[data['animal-type'] == 'Wolf']

# Define grid search parameters
learning_rates = [1e-5, 1e-4, 1e-3]
nodes = [4, 16, 64, 256]
layers = [3, 5, 7]
batch_size = 32
epochs = 10
patience = 3
n_lags = 3  # Number of lagged observations to use

# Prepare a dictionary to store the results for each wolf
results = {}

# Function to create lagged features
def create_lagged_features(data, n_lags):
    X, Y = [], []
    for i in range(n_lags, len(data)):
        X.append(data.iloc[i-n_lags:i].values.flatten())
        Y.append(data.iloc[i].values)
    return np.array(X), np.array(Y)

# Process each wolf's data
for wolf_id, group in data.groupby('individual-id'):
    print(f"Processing wolf {wolf_id}")
    
    X, y = create_lagged_features(group[['location-lat', 'location-long']], n_lags)
    
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y)
    
    # Split the data
    n_train = int(0.8 * len(X_scaled))
    X_train_scaled, y_train_scaled = X_scaled[:n_train], y_scaled[:n_train]
    X_test_scaled, y_test_scaled = X_scaled[n_train:], y_scaled[n_train:]

    # Initialize the best score to some high value
    best_score = np.inf

    # Grid search
    for lr in learning_rates:
        for node in nodes:
            for layer in layers:
                # Define the model
                model = Sequential()
                model.add(Dense(node, input_dim=n_lags*2, activation='relu'))  # Input dimension based on lagged features
                for _ in range(layer - 1):
                    model.add(Dense(node, activation='relu'))
                model.add(Dense(2, activation='linear'))  # Output layer
                
                # Compile the model
                optimizer = Adam(learning_rate=lr)
                model.compile(loss='mse', optimizer=optimizer)
                
                # Train the model with EarlyStopping
                early_stopping = EarlyStopping(monitor='val_loss', patience=patience)
                history = model.fit(
                    X_train_scaled, y_train_scaled, 
                    validation_split=0.2,  # Using 20% of the training data for validation
                    epochs=epochs, 
                    batch_size=batch_size, 
                    callbacks=[early_stopping], 
                    verbose=0
                )

                # Evaluate the model
                score = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
                print(f"Model with LR: {lr}, Nodes: {node}, Layers: {layer} has MSE: {score}")

                # Update best model if improved
                if score < best_score:
                    best_score = score
                    best_params = {'lr': lr, 'nodes': node, 'layers': layer}
                    best_model = model

    # Forecast future values using the last available input from training
    current_input = X_train_scaled[-1:].reshape(1, -1)  # Reshape for single prediction input
    predictions = []
    for _ in range(len(X_test_scaled)):
        next_prediction = best_model.predict(current_input)
        predictions.append(next_prediction)
        current_input = np.roll(current_input, -2)
        current_input[0, -2:] = next_prediction


    # Transform predictions back to original scale
    predictions_scaled_back = scaler_y.inverse_transform(np.vstack(predictions))
    mse = mean_squared_error(y_test_scaled, predictions_scaled_back)

    results[wolf_id] = {
        'mse': mse,
        'predictions': predictions_scaled_back,
        'actual': scaler_y.inverse_transform(y_test_scaled),
        'best_params': best_params
    }
    
# Output the results for each wolf
for wolf_id, res in results.items():
    print(f"Wolf ID: {wolf_id}, MSE: {res['mse']}, Best Params: {res['best_params']}")


##### Print best model for each wolf (with MSE on test set)

In [95]:
for wolf_id, res in results.items():
    if 'best_params' in res:
        print(f"Wolf ID: {wolf_id}")
        print(f"Best Parameters: Learning Rate: {res['best_params']['lr']}, Nodes: {res['best_params']['nodes']}, Layers: {res['best_params']['layers']}")
        print(f"Mean Squared Error (MSE) on Test Set: {res['mse']}")
        print("-----")

##### Save the best models, results and scaler

In [88]:
import pickle
from tensorflow.keras.models import load_model

# Save the best model for each wolf in the Keras format
best_model.save(f'best_model_for_wolf_{wolf_id}.keras')

# Save results to a pickle file
with open('results.pickle', 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Save the scalers for X and y 
    with open('scaler_X.pkl', 'wb') as f:
        pickle.dump(scaler_X, f)
    with open('scaler_y.pkl', 'wb') as f:
        pickle.dump(scaler_y, f)


##### Load the best models, results, and scaler

In [93]:
# Load the model with a .keras extension
loaded_model = load_model(f'best_model_for_wolf_{wolf_id}.keras')

# Load the results
with open('results.pickle', 'rb') as handle:
    loaded_results = pickle.load(handle)

# Load the scalers
with open('scaler_X.pkl', 'rb') as f:
    loaded_scaler_X = pickle.load(f)

with open('scaler_y.pkl', 'rb') as f:
    loaded_scaler_y = pickle.load(f)


#### Map of prediction

In [64]:
import folium
import pandas as pd

# Function to create a map for a single wolf
def create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id):
    # Starting point for the map
    start_lat = wolf_data['location-lat'].iloc[0]
    start_long = wolf_data['location-long'].iloc[0]
    wolf_map = folium.Map(location=[start_lat, start_long], zoom_start=8)

    # Split the data into training and testing
    split_index = int(len(wolf_data) * 0.8)
    train_data = wolf_data.iloc[:split_index]
    test_data = wolf_data.iloc[split_index:]

    # Define the HTML code for a purple cross
    html_purple_cross = '''
    <div style="position: relative; width: 8px; height: 8px;">
        <div style="position: absolute; top: 50%; left: 0; transform: translate(0%, -50%); width: 100%; height: 2px; background-color: purple;"></div>
        <div style="position: absolute; top: 0; left: 50%; transform: translate(-50%, 0%); width: 2px; height: 100%; background-color: purple;"></div>
    </div>
    '''

    # Add training data points
    train_points = [(row['location-lat'], row['location-long']) for idx, row in train_data.iterrows()]
    folium.PolyLine(train_points, color="blue", weight=2.5, opacity=1).add_to(wolf_map)
    for point in train_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='blue',
            fill=True,
            fill_color='blue',
            popup='Train'
        ).add_to(wolf_map)

    # Add test data points
    test_points = [(row['location-lat'], row['location-long']) for idx, row in test_data.iterrows()]
    folium.PolyLine(test_points, color="green", weight=2.5, opacity=1).add_to(wolf_map)
    for point in test_points:
        folium.CircleMarker(
            location=point,
            radius=3,
            color='green',
            fill=True,
            fill_color='green',
            popup='Test'
        ).add_to(wolf_map)

    # Mark the first test observation with a purple cross
    folium.Marker(
        location=test_points[0],
        icon=folium.DivIcon(html=html_purple_cross),
        popup='First Test Observation'
    ).add_to(wolf_map)

    # Add prediction points and draw lines between them
    prediction_points = list(zip(test_preds_lat, test_preds_long))
    folium.PolyLine(prediction_points, color="red", weight=2.5, opacity=1).add_to(wolf_map)
    for idx, point in enumerate(prediction_points):
        folium.CircleMarker(
            location=point,
            radius=3,
            color='red',
            fill=True,
            fill_color='red',
            popup=f'Predicted: {idx}'
        ).add_to(wolf_map)

    # Save the map in a specific directory for FNN results
    wolf_map.save(f'../Visualisation/FNN/wolf_{wolf_id}_map.html')

# Iterate through each wolf and create a map
for wolf_id, info in results.items():
    test_preds_lat = info['predictions'][:, 0]  
    test_preds_long = info['predictions'][:, 1]
    wolf_data = data[data['individual-id'] == wolf_id]
    create_wolf_map(wolf_data, test_preds_lat, test_preds_long, wolf_id)


#### Compute the RMSE, MAE and Haversine metrics

In [83]:
print(results)

{'B042': {'training_accuracy': {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}, 'test_accuracy': {'Mean_Haversine_Distance': 27.132738455889378, 'Median_Haversine_Distance': 26.379128873400788}}, 'B045': {'training_accuracy': {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}, 'test_accuracy': {'Mean_Haversine_Distance': 31.7936922809775, 'Median_Haversine_Distance': 31.596251955466627}}, 'B065': {'training_accuracy': {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}, 'test_accuracy': {'Mean_Haversine_Distance': 42.087636286319, 'Median_Haversine_Distance': 41.4538188010044}}, 'B077': {'training_accuracy': {'MAE_lat': 0.1533827259899431, 'RMSE_lat': 0.17944551606835518, 'MAE_long': 0.1603340323615948, 'RMSE_long': 0.18305228990857966}, 'test_accu

In [90]:
import numpy as np
from math import radians, cos, sin, asin, sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great-circle distance between two points 
    on the Earth (specified in decimal degrees).
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers. Adjust if using a different radius
    return c * r

for wolf_id, res in results.items():
    test_data = res['actual']
    predictions = res['predictions']
    
    # Extract latitude and longitude from predictions and actual data
    lat_preds, lon_preds = predictions[:, 0], predictions[:, 1]
    lat_actual, lon_actual = test_data[:, 0], test_data[:, 1]

    # Calculate the Haversine distance for each prediction
    distances = [haversine(lon_p, lat_p, lon_a, lat_a) for lon_p, lat_p, lon_a, lat_a in zip(lon_preds, lat_preds, lon_actual, lat_actual)]
    
    # Calculate metrics
    mae_lat = mean_absolute_error(lat_actual, lat_preds)
    rmse_lat = sqrt(mean_squared_error(lat_actual, lat_preds))
    mae_lon = mean_absolute_error(lon_actual, lon_preds)
    rmse_lon = sqrt(mean_squared_error(lon_actual, lon_preds))
    
    mean_distance = np.mean(distances)
    median_distance = np.median(distances)
    
    # Store extended metrics in the results dictionary
    res.update({
        'MAE_Latitude': mae_lat,
        'RMSE_Latitude': rmse_lat,
        'MAE_Longitude': mae_lon,
        'RMSE_Longitude': rmse_lon,
        'Mean_Haversine_Distance': mean_distance,
        'Median_Haversine_Distance': median_distance
    })

# Print the metrics for each wolf
for wolf_id, info in results.items():
    print(f"Wolf ID: {wolf_id}")
    print(f"MAE Latitude: {info['MAE_Latitude']:.4f}")
    print(f"RMSE Latitude: {info['RMSE_Latitude']:.4f}")
    print(f"MAE Longitude: {info['MAE_Longitude']:.4f}")
    print(f"RMSE Longitude: {info['RMSE_Longitude']:.4f}")
    print(f"Mean Haversine Distance: {info['Mean_Haversine_Distance']:.2f} km")
    print(f"Median Haversine Distance: {info['Median_Haversine_Distance']:.2f} km")
    print("-----")


KeyError: 'actual'

## 4. LSTM 

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = '../Data/merged_data.csv'
data = pd.read_csv(file_path)

# Separate wolf and elk data
wolf_data = data[data['animal-type'] == 'Wolf'].sort_values('timestamp')
elk_data = data[data['animal-type'] == 'Elk'].sort_values('timestamp')

# Features: Elk locations, Target: Wolf locations
features = elk_data[['location-lat', 'location-long']].values
target = wolf_data[['location-lat', 'location-long']].values

# Normalize the data
scaler_features = MinMaxScaler(feature_range=(0, 1))
scaler_target = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler_features.fit_transform(features)
target_scaled = scaler_target.fit_transform(target)

# Define time_steps and prepare the data sequences
time_steps = 50
x_final, y_final = [], []
for k in range(len(features_scaled) - time_steps):
    x_final.append(features_scaled[k:k + time_steps])
    y_final.append(target_scaled[k + time_steps])

x_final = np.array(x_final)
y_final = np.array(y_final)

# SHUFFLE THE DATA

# Split the data
train_features, test_features, train_target, test_target = train_test_split(x_final, y_final, test_size=0.2, shuffle=False)

# LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(time_steps, 2), return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(2)
])

# Compile and fit the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mse')
history = model.fit(train_features, train_target, epochs=10, validation_data=(test_features, test_target), verbose=1)

# Model summary
model.summary()


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer

# Load the dataset
file_path = '../Data/merged_data.csv'
data = pd.read_csv(file_path)

# Align and merge the data based on rounded timestamps
merged_data['rounded_timestamp'] = merged_data['timestamp'].dt.round('H')  # Adjust the rounding as needed
wolf_data = merged_data[merged_data['animal-type'] == 'Wolf']
elk_data = merged_data[merged_data['animal-type'] == 'Elk']

# Merge wolf and elk data on rounded_timestamp
aligned_data = pd.merge(wolf_data, elk_data, on='rounded_timestamp', suffixes=('_wolf', '_elk'))

# Handle missing values
imputer = SimpleImputer(strategy='ffill')
aligned_data[['location-lat_elk', 'location-long_elk']] = imputer.fit_transform(aligned_data[['location-lat_elk', 'location-long_elk']])

# Prepare the feature and target matrices
features = aligned_data[['location-lat_elk', 'location-long_elk']].values
target = aligned_data[['location-lat_wolf', 'location-long_wolf']].values

# Normalize the data
scaler_features = MinMaxScaler(feature_range=(0, 1))
scaler_target = MinMaxScaler(feature_range=(0, 1))
features_scaled = scaler_features.fit_transform(features)
target_scaled = scaler_target.fit_transform(target)

# Define time_steps and prepare the data sequences
time_steps = 50
x_final, y_final = [], []
for k in range(len(features_scaled) - time_steps):
    x_final.append(features_scaled[k:k + time_steps])
    y_final.append(target_scaled[k + time_steps])

x_final = np.array(x_final)
y_final = np.array(y_final)

# Split the data into training and test sets
train_features, test_features, train_target, test_target = train_test_split(x_final, y_final, test_size=0.2, shuffle=False)

# Define the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(64, input_shape=(time_steps, 2), return_sequences=True),
    tf.keras.layers.LSTM(32, activation='relu'),
    tf.keras.layers.Dense(2)  # Output layer for latitude and longitude
])

# Compile and fit the model
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='mse')
history = model.fit(train_features, train_target, epochs=10, validation_data=(test_features, test_target), verbose=1)

# Model summary
model.summary()


### LSTM TEST

In [ ]:
import pandas as pd

# Load the dataset
file_path = '../Data/merged_data.csv'
data = pd.read_csv(file_path)

##### Create the LSTM model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

# Convert timestamps to datetime and sort the data chronologically
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.sort_values('timestamp', inplace=True)

# Selecting latitude and longitude as features for the LSTM model
features = data[['location-lat', 'location-long']].values

# Normalize the features using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# Define the number of timestamps to use in the input sequence
sequence_length = 3

# Split the data into training and test sets (80% train, 20% test)
train_features, test_features = train_test_split(scaled_features, test_size=0.2, shuffle=False)

# Create the TimeseriesGenerator for training and test sets
train_generator = TimeseriesGenerator(train_features, train_features, length=sequence_length, batch_size=1)
test_generator = TimeseriesGenerator(test_features, test_features, length=sequence_length, batch_size=1)

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(sequence_length, 2), return_sequences=False))
model.add(Dense(units=2))  # Output layer with 2 units for latitude and longitude

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')

# Fit the model to the data
history = model.fit(train_generator, epochs=1, verbose=1, validation_data=test_generator)

# Summary of the model architecture
model.summary()


##### Predictions for the first wolf with the model + Haversine accuracy

In [ ]:
# Filter data for the first wolf
first_wolf_data = data[data['individual-local-identifier'] == data['individual-local-identifier'].unique()[0]]

# Extracting relevant features and scaling
first_wolf_features = scaler.transform(first_wolf_data[['location-lat', 'location-long']].values)

# Generating sequences for the first wolf
first_wolf_sequences = TimeseriesGenerator(first_wolf_features, first_wolf_features,
                                           length=sequence_length, batch_size=1)

# Prepare containers for the predictions and actual values
predictions = []
actual = []

# Generate predictions for each sequence
for i in range(len(first_wolf_sequences)):
    x, y = first_wolf_sequences[i]
    yhat = model.predict(x)
    predictions.append(yhat[0])
    actual.append(y[0])

# Inverse the scaling
predictions = scaler.inverse_transform(predictions)
actual = scaler.inverse_transform(actual)

# Calculate Haversine distance for each prediction
distances = [haversine((act[0], act[1]), (pred[0], pred[1])) for act, pred in zip(actual, predictions)]

# Calculate the mean distance error
mean_distance_error = sum(distances) / len(distances)

# Output the result
print(f'Mean Haversine Distance Error for the first wolf: {mean_distance_error} kilometers')


##### Computing MAE and RMSE accuracy

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse_lat = sqrt(mean_squared_error(actual[:, 0], predictions[:, 0]))
rmse_long = sqrt(mean_squared_error(actual[:, 1], predictions[:, 1]))
mae_lat = mean_absolute_error(actual[:, 0], predictions[:, 0])
mae_long = mean_absolute_error(actual[:, 1], predictions[:, 1])

print(f'Latitude RMSE: {rmse_lat}')
print(f'Longitude RMSE: {rmse_long}')
print(f'Latitude MAE: {mae_lat}')
print(f'Longitude MAE: {mae_long}')
